To run a cell, type `Shift` + `Return`. Run the cell below to get started

In [ ]:
from workshop_utils import * 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns
import datetime
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Buildings Example

_Tacloban, Philippines_

These next cells will load the output from the following query in Athena:

```sql
SELECT date_trunc('hour', updated), avg( date_diff('minute', updated, valid_until) ) AS avg_lifespan_minutes, count(distinct(changesets.uid)) AS num_users, count(*) AS new_buildings
FROM southeastern_asia
JOIN changesets on southeastern_asia.changeset = changesets.id
WHERE ST_WITHIN( geom , ST_Polygon('POLYGON((124.909271 11.293189, 125.088518 11.293189, 125.088518 10.954934, 124.909271 10.954934, 124.909271 11.293189))') )
        AND element_at(southeastern_asia.tags, 'building') IS NOT NULL
        AND version = 1
        AND minor_version < 1
GROUP BY  date_trunc('hour', updated)
```

In [ ]:
#Tacloban, Philippines
tacloban = load_dataframe_from_s3('https://us-east-2.console.aws.amazon.com/athena/query/results/5d020663-b325-49ac-930e-5bdd4ee88b39/csv')


In [ ]:
tacloban.head(2)

In [ ]:
#Rename the columns so they make more sense
tacloban.columns = ["datetime", "lifespan_minutes", "num_users", "objects"]

#Cast the date as date
tacloban['datetime'] = tacloban.datetime.apply(pd.Timestamp)

#Reindex (and sort)by date becuase the original query was organized that way
tacloban.set_index('datetime', inplace=True)
tacloban.sort_index(inplace=True)

#Add two new columns:
tacloban['lifespan_days'] = tacloban.lifespan_minutes.apply(lambda x: x/(60*24))
tacloban['object_percent'] = tacloban.objects / tacloban.objects.sum()

#Finally, show the new dataframe
tacloban.head(4)

In [ ]:
sns.set_style('whitegrid')
ax = tacloban['objects'].plot(style='.', figsize=(14,4))
# tacloban['objects'].plot(ax=ax)
ax.set_xlabel("Date"); ax.set_ylabel("");
ax.set_title("When buildings were added in Tacloban, Philippines",fontsize=16);

In [ ]:
sns.set_style('whitegrid')
fig, ax = plt.subplots(figsize=(15,7))
tacloban['objects'].cumsum().plot(figsize=(14,4), ax=ax)
ax.set_xlabel("Date"); ax.set_ylabel("");
ax.set_title("Impact of Typhoon Yolanda on Buildings in Tacloban",fontsize=16);
ax.set_xlim([datetime.date(2013,1,1), datetime.date(2015,1,1)])
#set ticks every day
ax.xaxis.set_major_locator(mdates.MonthLocator())
#set major ticks format
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

In [ ]:
# These buildings were the work of how many users?
ax = tacloban['num_users'].plot(style=".", figsize=(14,4))
ax.set_xlabel("Date"); ax.set_ylabel(""); 
ax.set_title("Users adding new buildings in Tacloban, Philippines",fontsize=16);
# ax.set_xlim([datetime.date(2013,11,1), datetime.date(2013,11,30)]);


In [ ]:
print("The number of buildings in Tacloban is: {:,}".format(tacloban.objects.sum()))

## Now retrieve all of the actual buildings from Athena 


```sql
SELECT southeastern_asia.id AS id,
         southeastern_asia.type AS type,
         southeastern_asia.geom AS geom,
         southeastern_asia.tags AS b_tags,
         southeastern_asia.changeset,
         southeastern_asia.updated,
         southeastern_asia.valid_until,
         southeastern_asia.version,
         southeastern_asia.minor_version,
         changesets.id,
         changesets.uid,
         changesets.user,
         changesets.tags AS c_tags
FROM southeastern_asia
JOIN changesets
    ON southeastern_asia.changeset = changesets.id
WHERE ST_WITHIN( geom , ST_Polygon('POLYGON((124.909271 11.293189, 125.088518 11.293189, 125.088518 10.954934, 124.909271 10.954934, 124.909271 11.293189))') )
        AND element_at(southeastern_asia.tags, 'building') IS NOT NULL
```

In [ ]:
buildings = load_dataframe_from_s3('https://us-east-2.console.aws.amazon.com/athena/query/results/77664a61-e7b4-4994-a09a-7546d93cb488/csv')


In [ ]:
buildings.head(2)

In [ ]:
# Cast timestamp to a date
buildings['date'] = buildings.updated.apply(lambda x: pd.Timestamp(x).date())

# How about the edit lifespan (time between edits)
buildings['lifespan'] = buildings.apply(lambda row: pd.Timestamp(row.valid_until) - pd.Timestamp(row.updated), axis=1)

In [ ]:
ax = buildings.groupby('date').aggregate('count')['id'].plot(style=".", figsize=(14,4))
ax.set_title("Edits to buildings each day in Tacloban, Philippines",fontsize=16);
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'));

### `Minor Version`

We can use the `minor version` attribute to identify geometry updates to buildings. This can be a form of validation behavior. identifying spikes in the occurence of these can identify map validation.

In [ ]:
ax = buildings[buildings.minor_version>0].groupby('date').aggregate('count')['id'].plot(figsize=(14,4))
ax.set_title("Adjustments to building geometries each day in Tacloban, Philippines",fontsize=16);

What is the most edited building? 

In [ ]:
# This is a bit hacky, but it definitely works
most_edited_building = pd.DataFrame(buildings[buildings.id==buildings.sort_values(
    by='version',ascending=False).id.values[0]].sort_values(
      by=['version','minor_version'],ascending=False))

print("most edited building: http://openstreetmap.org/way/{}\nThe most recent three edits:".format(most_edited_building.id.values[0]))

most_edited_building.head(3)

In [ ]:
# Extending that, which buildings were edited by the most people?

buildings.groupby('id').aggregate({
    'uid':pd.Series.nunique,
    'version':pd.Series.max,
    'lifespan': pd.Series.mean
}).sort_values(by='uid',ascending=False).head()

# For comparison, look at Pokhara, Nepal:

```sql
SELECT date_trunc('day', updated), avg( date_diff('minute', updated, valid_until) ) AS lifespan_minutes, count(distinct(changeset)) AS changesets, count(*) AS objects
FROM nepal
WHERE ST_WITHIN( geom , ST_Polygon('POLYGON((83.783947 28.38623, 84.182564 28.38623, 84.182564 28.055363, 83.783947 28.055363, 83.783947 28.38623))') )
        AND element_at(nepal.tags, 'building') IS NOT NULL
        AND version = 1
        AND minor_version < 1
GROUP BY  date_trunc('day', updated)
```

In [ ]:
# The above query was already run, results are here: 
df = load_dataframe_from_s3("https://us-east-2.console.aws.amazon.com/athena/query/results/83947f62-d3be-4aa5-8f76-fd1c2ef1ef67/csv")


In [ ]:
#What does the data look like?
df.head(2)

Rename columns and calculate a few new ones:

In [ ]:
#Rename the columns so they make more sense
df.columns = ["date", "lifespan_minutes", "changesets", "objects"]

#Cast the date as date
df.date = df.date.apply(pd.Timestamp)

#Reindex (and sort)by date becuase the original query was organized that way
df.set_index('date', inplace=True)
df.sort_index(inplace=True)

#Add two new columns:
df['lifespan_days'] = df.lifespan_minutes.apply(lambda x: x/(60*24))
df['object_percent'] = df.objects / df.objects.sum()

#Finally, show the new dataframe
df.head(4)

In [ ]:
ax = df['objects'].plot(figsize=(14,4))
ax.set_xlabel("Date"); ax.set_ylabel(""); ax.set_title("When buildings were added in Pokhara, Nepal",fontsize=16);

While we see the Earthquake in April 2015 has a major effect on the number of new buildings, we {

The majority of the buildings in Pokhara, Nepal were not added after the earthquake. Instead, a year after the earthquake saw the largest building mapping efforts.

In [ ]:
ax = df['objects'].cumsum().plot(figsize=(14,4))
ax.set_xlabel("Date"); ax.set_ylabel(""); ax.set_title("When buildings were added in Pokhara, Nepal",fontsize=16);

Direct comparison of the volume of buildings added to the map, annotated:

In [ ]:
ax = df['objects'].cumsum().plot(figsize=(14,4), label="Pokhara, Nepal")
tacloban['objects'].cumsum().plot(ax=ax, label="Tacloban, Phillipines")
ax.set_xlabel("Date"); ax.set_ylabel(""); ax.set_title("When buildings were added (not edited) in Philippines & Nepal",fontsize=16);
ax.legend(fontsize=16);
ax.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

ax.vlines(datetime.datetime(2013,11,8),0,180000, color='gray',alpha=0.75, linestyle="--");
ax.vlines(datetime.datetime(2015,4,25),0,180000,color='gray',alpha=0.75, linestyle="--");

ax.text(x=datetime.datetime(2013,11,15),y=125000,s='Typhoon\nHaiyan', fontsize=12);
ax.text(x=datetime.datetime(2015,5,1),y=125000,s='Nepal\nEarthquake', fontsize=12);
